In [2]:
from google.colab import files
import os

print("Please upload orders.csv, users.json, and restaurants.sql one by one or all at once:")
uploaded =files.upload()

# Verification to make sure they are there
for filename in ['orders.csv', 'users.json', 'restaurants.sql']:
    if filename in os.listdir():
        print(f" {filename} uploaded successfully.")
    else:
        print(f"{filename} is missing. Please upload it.")

Please upload orders.csv, users.json, and restaurants.sql one by one or all at once:


Saving orders.csv to orders (1).csv
Saving restaurants.sql to restaurants.sql
Saving users.json to users.json
 orders.csv uploaded successfully.
 users.json uploaded successfully.
 restaurants.sql uploaded successfully.


In [3]:
import pandas as pd
import json
import re

# 1. Load CSV Data
orders_df = pd.read_csv('orders.csv')

# 2. Load JSON Data
with open('users.json', 'r') as f:
    users_data = json.load(f)
users_df = pd.DataFrame(users_data)

# 3. Load SQL Data (Parsing the INSERT statements)
with open('restaurants.sql', 'r') as f:
    sql_content = f.read()
# This pattern matches the (id, 'name', 'cuisine', rating) format in your SQL file
pattern = r"\((\d+),\s*'([^']*)',\s*'([^']*)',\s*([\d\.]+)\)"
matches = re.findall(pattern, sql_content)
restaurants_df = pd.DataFrame(matches, columns=['restaurant_id', 'restaurant_name_sql', 'cuisine', 'rating'])
restaurants_df['restaurant_id'] = restaurants_df['restaurant_id'].astype(int)
restaurants_df['rating'] = restaurants_df['rating'].astype(float)

# 4. Merge the Data
# Using Left Join to retain all orders as per instructions
final_df = orders_df.merge(users_df, on='user_id', how='left')
final_df = final_df.merge(restaurants_df, on='restaurant_id', how='left')

# Save the master dataset
final_df.to_csv('final_food_delivery_dataset.csv', index=False)
print("✅ Final dataset created and saved as 'final_food_delivery_dataset.csv'")

✅ Final dataset created and saved as 'final_food_delivery_dataset.csv'


In [4]:
# Convert date to datetime objects
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)

print("--- HACKATHON ANALYSIS RESULTS ---")

# 1. Highest revenue city for Gold members
gold_df = final_df[final_df['membership'] == 'Gold']
top_city = gold_df.groupby('city')['total_amount'].sum().idxmax()
print(f"1. Highest Revenue City (Gold): {top_city}")

# 2. Total Gold Orders
print(f"2. Total Orders by Gold Members: {len(gold_df)}")

# 3. Average Order Value (Gold)
avg_gold = gold_df['total_amount'].mean()
print(f"3. Avg Order Value (Gold): {avg_gold:.2f}")

# 4. Total Hyderabad Revenue (Rounded)
hyd_rev = final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()
print(f"4. Total Hyderabad Revenue: {round(hyd_rev)}")

# 5. Distinct Users
print(f"5. Total Distinct Users: {final_df['user_id'].nunique()}")

# 6. High Rating Orders (>= 4.5)
high_rating = final_df[final_df['rating'] >= 4.5]
print(f"6. Orders with Restaurant Rating >= 4.5: {len(high_rating)}")

# 7. Highest revenue combo (Check specific options)
combo_rev = final_df.groupby(['membership', 'cuisine'])['total_amount'].sum().idxmax()
print(f"7. Highest Revenue Combo: {combo_rev}")

--- HACKATHON ANALYSIS RESULTS ---
1. Highest Revenue City (Gold): Chennai
2. Total Orders by Gold Members: 4987
3. Avg Order Value (Gold): 797.15
4. Total Hyderabad Revenue: 1889367
5. Total Distinct Users: 2883
6. Orders with Restaurant Rating >= 4.5: 3374
7. Highest Revenue Combo: ('Regular', 'Mexican')


In [5]:
import pandas as pd
import json
import re

# 1. Load CSV
orders_df = pd.read_csv('orders.csv')

# 2. Load JSON
with open('users.json', 'r') as f:
    users_df = pd.DataFrame(json.load(f))

# 3. Load SQL (Parsing)
with open('restaurants.sql', 'r') as f:
    sql_content = f.read()
pattern = r"\((\d+),\s*'([^']*)',\s*'([^']*)',\s*([\d\.]+)\)"
matches = re.findall(pattern, sql_content)
restaurants_df = pd.DataFrame(matches, columns=['restaurant_id', 'restaurant_name_sql', 'cuisine', 'rating'])
restaurants_df['restaurant_id'] = restaurants_df['restaurant_id'].astype(int)
restaurants_df['rating'] = restaurants_df['rating'].astype(float)

# 4. Merge Data (Left Join to keep all orders)
final_df = orders_df.merge(users_df, on='user_id', how='left')
final_df = final_df.merge(restaurants_df, on='restaurant_id', how='left')

# Pre-processing
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)
final_df.to_csv('final_food_delivery_dataset.csv', index=False)
print("Dataset Merged Successfully. Total Rows:", len(final_df))

Dataset Merged Successfully. Total Rows: 10000


In [6]:
print("--- MCQ ANSWERS ---")

# Q1: Highest Revenue City for Gold Members
gold_members = final_df[final_df['membership'] == 'Gold']
print("1. Top Gold City:", gold_members.groupby('city')['total_amount'].sum().idxmax())

# Q2: Highest Avg Order Value Cuisine
print("2. Top Avg Cuisine:", final_df.groupby('cuisine')['total_amount'].mean().idxmax())

# Q3: Distinct Users > 1000 Total
user_revenue = final_df.groupby('user_id')['total_amount'].sum()
print("3. Users > 1000 Total:", (user_revenue > 1000).sum())

# Q4: Rating Range Revenue
def rating_range(r):
    if 3.0 <= r <= 3.5: return '3.0 – 3.5'
    if 3.6 <= r <= 4.0: return '3.6 – 4.0'
    if 4.1 <= r <= 4.5: return '4.1 – 4.5'
    if 4.6 <= r <= 5.0: return '4.6 – 5.0'
final_df['rating_range'] = final_df['rating'].apply(rating_range)
print("4. Top Revenue Rating Range:", final_df.groupby('rating_range')['total_amount'].sum().idxmax())

# Q7: Percentage of Gold Orders
gold_pct = (len(gold_members) / len(final_df)) * 100
print(f"7. Gold Order %: {round(gold_pct)}%")

# Q9: Combination Revenue (Specific options)
combo = final_df.groupby(['membership', 'cuisine'])['total_amount'].sum()
print("9. Gold + Italian Revenue:", combo.get(('Gold', 'Italian')))

# Q10: Quarter Revenue
final_df['quarter'] = final_df['order_date'].dt.quarter
print("10. Top Revenue Quarter: Q", final_df.groupby('quarter')['total_amount'].sum().idxmax())

--- MCQ ANSWERS ---
1. Top Gold City: Chennai
2. Top Avg Cuisine: Mexican
3. Users > 1000 Total: 2544
4. Top Revenue Rating Range: 4.6 – 5.0
7. Gold Order %: 50%
9. Gold + Italian Revenue: 1005779.05
10. Top Revenue Quarter: Q 3


In [7]:
print("--- NUMERICAL ANSWERS ---")

# 1. Total Gold Orders
print("1. Total Gold Orders:", len(gold_members))

# 2. Hyderabad Total Revenue
hyd_rev = final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()
print("2. Hyderabad Revenue (Rounded):", round(hyd_rev))

# 3. Distinct Users
print("3. Total Distinct Users:", final_df['user_id'].nunique())

# 4. Avg Order Value (Gold)
print("4. Avg Gold Order Value:", round(gold_members['total_amount'].mean(), 2))

# 5. Rating >= 4.5 Orders
print("5. Orders (Rating >= 4.5):", len(final_df[final_df['rating'] >= 4.5]))

# 6. Orders in Top Gold City (Chennai)
top_gold_city = gold_members.groupby('city')['total_amount'].sum().idxmax()
print(f"6. Gold Orders in {top_gold_city}:", len(gold_members[gold_members['city'] == top_gold_city]))

--- NUMERICAL ANSWERS ---
1. Total Gold Orders: 4987
2. Hyderabad Revenue (Rounded): 1889367
3. Total Distinct Users: 2883
4. Avg Gold Order Value: 797.15
5. Orders (Rating >= 4.5): 3374
6. Gold Orders in Chennai: 1337


In [8]:
print("--- FILL IN THE BLANK CHECKS ---")
print("Join Key (Orders/Users): user_id")
print("Join Key (Orders/Restaurants): restaurant_id")
print("Total Rows in Final Data:", len(final_df))
print("Missing Value Representation: NaN/Null")

--- FILL IN THE BLANK CHECKS ---
Join Key (Orders/Users): user_id
Join Key (Orders/Restaurants): restaurant_id
Total Rows in Final Data: 10000
Missing Value Representation: NaN/Null
